<a href="https://colab.research.google.com/github/ClauCll/Curso_ML_A_Z/blob/master/Regresi%C3%B3n/borrador.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Cuantas serán las bajas voluntarias de los clientes fijos para un periodo x. (Agosto - Diciembre 2020)
 

In [247]:
import pandas as pd
import numpy as np

#Gráfico
from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt 
from matplotlib import style
import matplotlib.ticker as ticker
import seaborn as sns

import gspread
from oauth2client.client import GoogleCredentials 
from google.colab import auth
# Autoriza Claudia
auth.authenticate_user()
gc = gspread.authorize (GoogleCredentials.get_application_default())

### leemos datos

In [248]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [249]:
data = pd.read_csv('/content/drive/My Drive/Dataset/M_BAJAS.txt',delimiter='~')
data.Fecha = pd.to_datetime(data.FECHA_BAJA)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  


In [ ]:
data.head(5)

In [250]:
encabezados= ['Año','Mes','Fecha','Antiguedad','Cant_Bajas']
data.columns=encabezados

In [ ]:
data.head(5)

In [ ]:
 #validamos datos faltantes
 print(pd.isnull(data).sum())

In [ ]:
data.describe(include='all')

In [ ]:
data.info 

In [251]:
#Agrupamos la columna antiguedad
#1 : 1 TRIMESTRE (0-93 DIAS)
#2 : 2 TRIMESTRE (94-186 DIAS)
#3 : 3 TRIMESTRE (187-279 DIAS)
#4 : 4 TRIMESTRE (280-370 DIAS)
#5 : A 2 AÑOS (371-725 DIAS) 
#6 : A 3 AÑOS (726-1095 DIAS)
#7 : 4 años en adelante (Máximo DIAS 7276) 
bins=[-1,93,186,279,370,725,1095,7276]
names=['1','2','3','4','5','6','7']
data['Antiguedad'] = pd.cut(data['Antiguedad'],bins,labels=names)

In [253]:
#validamos datos sin clasificar
data[pd.isnull(data.Antiguedad)]

,Año,Mes,Fecha,Antiguedad,Cant_Bajas


In [252]:
#pasamos datos a clasificados a entero
data['Antiguedad'] = data['Antiguedad'].astype('int64')

In [269]:
data1= data[data.Año == 2019 ] 
data2= data[data.Año == 2020 ] 

In [271]:
data.dtypes

Año            int64
Mes            int64
Fecha         object
Antiguedad     int64
Cant_Bajas     int64
dtype: object

In [268]:
data.head(5)

,Año,Mes,Fecha,Antiguedad,Cant_Bajas
0,2019,11,01/11/2019,6,3
1,2020,4,01/04/2020,6,1
2,2019,11,05/11/2019,7,1
3,2020,2,01/02/2020,7,1
4,2019,8,01/08/2019,6,1


In [ ]:
 #validamos datos faltantes
 print(pd.isnull(data).sum())

In [244]:
data1.drop(data1[data1.Año == 2020 ].index, inplace = True)

In [115]:
grupo = data.groupby(by = ['Mes','Antiguedad']).sum()
grupo= pd.DataFrame(grupo).reset_index()

In [116]:
grupo.head(5)

,Mes,Antiguedad,Cant_Bajas
0,1,1,47
1,1,2,44
2,1,3,45
3,1,4,212
4,1,5,1167


In [ ]:
 #validamos datos faltantes
 print(pd.isnull(grupo).sum())

In [ ]:
#grupo.dropna(subset = ['Cant_Bajas'] , axis=0, inplace=True)

In [180]:
x= grupo.iloc[:,:-1].values
x=np.array(x)
y= grupo.iloc[:,2].values
y = y[:,np.newaxis]

In [182]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

labelencoder = LabelEncoder ()
x [:, 1] = labelencoder.fit_transform (x [:, 1])
 
Oencoder = ColumnTransformer([('one_hot_encoder',OneHotEncoder(categories='auto'),[1])],remainder='passthrough')
x= Oencoder.fit_transform(x).toarray()

In [186]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2, random_state=0)

In [187]:
from sklearn.preprocessing import PolynomialFeatures
poly_reg = PolynomialFeatures(degree = 3)
x_poly_train = poly_reg.fit_transform(x_train)
x_poly_test = poly_reg.fit_transform(x_test)

from sklearn.linear_model import LinearRegression
lineal_poly = LinearRegression()
lineal_poly.fit(x_poly_train,y_train)

print("precisión del modelo regresión polinomial")
print(lineal_poly.score(x_poly_train,y_train))

precisión del modelo regresión polinomial
0.8782736505035478


In [188]:
y_predic_train = lineal_poly.predict(x_poly_train)
y_predic_test = lineal_poly.predict(x_poly_test)